In [2]:
# Añadir el directorio 'src' al sys.path
import sys
import os

In [3]:
# Obtener la ruta absoluta de la carpeta 'src' para que sea accesible desde el notebook
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [ ]:
# Importar la función de normalización
from preprocessing import normalize_text

<function normalize_text at 0x00000235D4E4B560>


In [5]:
# Importar pandas para cargar y manipular el dataset
import pandas as pd

In [6]:
# Cargar el dataset
df = pd.read_csv("../data/TA1C_dataset_detection_train.csv")


In [7]:
# Aplicar cada tipo de normalización
df['tokenized_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="tokenization"))
df['cleaned_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="text_cleaning"))
df['no_stopwords_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="remove_stopwords"))
df['lemmatized_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="lemmatization"))


In [8]:
# Mostrar ejemplos de cada tipo de normalización
print("\nEjemplos de normalización:")
print(df[['Teaser Text', 'tokenized_text', 'cleaned_text', 'no_stopwords_text', 'lemmatized_text']].head())



Ejemplos de normalización:
                                         Teaser Text  \
0  #SegundaDivisión  | La fortaleza del ataque: R...   
1  Jorge Lanata a los argentinos que se van a Uru...   
2  Raffo: “Los montevideanos deben estar alerta p...   
3  Ecos del universo: joven uruguayo desentraña (...   
4  Propuesta quinquenal de ANEP: aumento de 3,8% ...   

                                      tokenized_text  \
0  [#, SegundaDivisión,  , |, La, fortaleza, del,...   
1  [Jorge, Lanata, a, los, argentinos, que, se, v...   
2  [Raffo, :, “, Los, montevideanos, deben, estar...   
3  [Ecos, del, universo, :, joven, uruguayo, dese...   
4  [Propuesta, quinquenal, de, ANEP, :, aumento, ...   

                                        cleaned_text  \
0  segundadivisión   la fortaleza del ataque ramp...   
1  jorge lanata a los argentinos que se van a uru...   
2  raffo “los montevideanos deben estar alerta po...   
3  ecos del universo joven uruguayo desentraña y ...   
4  propuesta quinq

In [10]:

# Guardar el dataset con todas las columnas de normalización
output_path = "../data/TA1C_dataset_detection_train_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"Dataset con normalizaciones guardado en '{output_path}'")

Dataset con normalizaciones guardado en '../data/TA1C_dataset_detection_train_cleaned.csv'
